In [58]:
import pandas as pd

df = pd.read_csv('Anime_data.csv')
df

,Anime_id,Title,Genre,Synopsis,Type,Producer,Studio,Rating,ScoredBy,Popularity,Members,Episodes,Source,Aired,Link
0,1,Cowboy Bebop,"['Action', 'Adventure', 'Comedy', 'Drama', 'Sc...","In the year 2071, humanity has colonized sever...",TV,['Bandai Visual'],['Sunrise'],8.81,363889.0,39.0,704490.0,26.0,Original,"Apr 3, 1998 to Apr 24, 1999",https://myanimelist.net/anime/1/Cowboy_Bebop
1,5,Cowboy Bebop: Tengoku no Tobira,"['Action', 'Space', 'Drama', 'Mystery', 'Sci-Fi']","Another day, another bounty—such is the life o...",Movie,"['Sunrise', 'Bandai Visual']",['Bones'],8.41,111187.0,475.0,179899.0,1.0,Original,"Sep 1, 2001",https://myanimelist.net/anime/5/Cowboy_Bebop__...
2,6,Trigun,"['Action', 'Sci-Fi', 'Adventure', 'Comedy', 'D...","Vash the Stampede is the man with a $$60,000,0...",TV,['Victor Entertainment'],['Madhouse'],8.31,197451.0,158.0,372709.0,26.0,Manga,"Apr 1, 1998 to Sep 30, 1998",https://myanimelist.net/anime/6/Trigun
3,7,Witch Hunter Robin,"['Action', 'Magic', 'Police', 'Supernatural', ...",Witches are individuals with special powers li...,TV,['Bandai Visual'],['Sunrise'],7.34,31875.0,1278.0,74889.0,26.0,Original,"Jul 2, 2002 to Dec 24, 2002",https://myanimelist.net/anime/7/Witch_Hunter_R...
4,8,Bouken Ou Beet,"['Adventure', 'Fantasy', 'Shounen', 'Supernatu...",It is the dark century and the people are suff...,TV,NaN,['Toei Animation'],7.04,4757.0,3968.0,11247.0,52.0,Manga,"Sep 30, 2004 to Sep 29, 2005",https://myanimelist.net/anime/8/Bouken_Ou_Beet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16997,27499,Sore Ike! Anpanman: Anpanman to Kaizoku Lobster,NaN,NaN,Special,NaN,NaN,5.50,NaN,NaN,70.0,NaN,NaN,NaN,NaN
16998,12091,X Bomber,NaN,NaN,TV,NaN,NaN,5.94,NaN,NaN,199.0,NaN,NaN,NaN,NaN
16999,29133,X Bomber Pilot,NaN,NaN,Special,NaN,NaN,4.28,NaN,NaN,77.0,NaN,NaN,NaN,NaN
17000,34485,Ganko-chan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN


In [59]:
df.drop(['ScoredBy', 'Popularity','Members','Episodes','Source','Aired','Link','Anime_id'], axis=1, inplace=True)

In [60]:
df.dropna(thresh=5, axis=0, inplace=True)

In [61]:
 # one-hot encoding pour les variables categoriell
cat_list=["Type"]

df =pd.get_dummies(df, columns= cat_list)

In [62]:
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
import numpy as np 

def vectColumn(df, column_name):
    df[column_name].apply(lambda x: type(x)).value_counts()

    # Convertir les cellules en chaînes de caractères si nécessaire
    df[column_name] = df[column_name].astype(str)

    # Tokenize les données textuelles
    df[column_name] = df[column_name].apply(word_tokenize)

    model = Word2Vec(df[column_name], window=5, min_count=1)
    # Convert the words in the "Synopsis" column to vectors
    df[column_name] = df[column_name].apply(lambda x: [sum(model.wv[word])/len(x) for word in x])
 
        
    return df

In [63]:
df= vectColumn(df, 'Synopsis')

In [64]:
df= vectColumn(df, 'Title')

In [65]:
df['Title'] = df['Title'].apply(lambda x: np.array(x))
df['Synopsis'] = df['Synopsis'].apply(lambda x: np.array(x))

In [66]:
synopsis=df['Synopsis']
Title=df['Title']

In [67]:
import ast
import pandas as pd

def process_and_remove_column(df, column_name):
    processed_values = []

    # Iterate over each row in the dataframe
    for i, row in df.iterrows():
        # Get the values for the current row
        values = row[column_name]
    
        # Check if the value of values is not NaN
        if not pd.isnull(values):
            # Use ast.literal_eval to safely evaluate the string representation of the list
            values = ast.literal_eval(values)

            # Iterate over each value in the list of values
            for value in values:
                # Append the value to the processed_values list
                processed_values.append((i, value))
                
    processed_values = list(set(processed_values))
    # Create a new dataframe from the processed_values list
    df_processed = pd.DataFrame(processed_values, columns=['Index', column_name])

    # Pivot the new dataframe to create columns for each value
    df_pivot = df_processed.pivot(index='Index', columns=column_name, values=column_name)

    # Replace NaN values with 0
    df_pivot.fillna(0, inplace=True)

    # Convert the 0 values to False and the genre values to True
    df_pivot = df_pivot.astype(bool).astype(int)
    
    # Add the prefix to the column names to avoid conflicts
    if column_name == "Studio":
        prefix="studio_"
    elif column_name == "producer":
        prefix="producer_"
    else:
        prefix = "genre_"
    df_pivot.columns = [prefix + col for col in df_pivot.columns]

    # Join the pivot dataframe to the original dataframe
    df = df.join(df_pivot)

    # Drop the original column from the dataframe
    df = df.drop(column_name, axis=1)
    
  

    return df



In [68]:


df= process_and_remove_column(df, 'Studio')

In [69]:
df=process_and_remove_column(df, 'Genre')

In [70]:
df=process_and_remove_column(df, 'Producer')

In [71]:
df

,Title,Synopsis,Rating,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_Special,Type_TV,Type_Unknown,...,genre_feng,genre_flying DOG,genre_gimik,genre_i0+,genre_iQIYI,genre_indigo line,genre_m.o.e.,genre_project lights,genre_teamKG,genre_tsuritama partners
0,"[0.019310925796162337, -0.011677875027089613]","[0.00020464750237225257, 0.005315192563449676,...",8.81,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[0.006436975265387446, -0.003892625009029871, ...","[0.00959832645049773, -0.03554391090775181, -0...",8.41,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,[-0.009230552852386609],"[0.0010812541723046734, 0.005207814935905238, ...",8.31,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[0.01138263581863915, 0.035894299702097975, -0...","[0.005169035755071408, -0.15829741876487705, -...",7.34,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[0.009988797440504035, 0.015221555911314985, 0...","[-0.06922955825784871, -0.045365353410126145, ...",7.04,0,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15030,"[-0.0034692702029133216, 0.14585203697206453, ...","[0.0017630417576237093, 0.059579316792743545, ...",NaN,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15031,"[0.00932627295878774, 0.004464440162337269, 0....","[0.10645002868841402, 0.0021177566293317795, -...",5.29,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15037,"[0.006666698632761836, 0.05224411659874022, -0...","[0.011464242838943999, 0.02087888310973843, 0....",NaN,1,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15042,"[0.008931750229749014, 0.18975995294749737, 0....","[0.13591757319359618, 0.09161201885647395, 0.0...",6.91,0,0,1,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
import numpy as np 
corr = df.corr()
# Select highly correlated features
high_corr = corr[((corr > 0.05) | (corr < -0.05)) & (corr < 1) ]
# For each feature, list its correlated feature
correlated_columns = {}
for col in high_corr.columns:
    correlated_features = high_corr.columns[(~high_corr[col].isna())].tolist()
    correlated_features = list(set(correlated_features).difference(set(correlated_columns.keys())))
    correlated_columns[col] = correlated_features

correlated_columns

C:\Users\hp\AppData\Local\Temp\ipykernel_13800\582176351.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


{'Rating': ['studio_Pie in The Sky',
  'genre_Kids',
  'genre_Movic',
  'genre_Kadokawa Pictures Japan',
  'genre_Avex Pictures',
  'studio_A-1 Pictures',
  'studio_dwarf',
  'studio_Asahi Production',
  'genre_Starchild Records',
  'genre_Asmik Ace Entertainment',
  'studio_Wit Studio',
  'genre_Shoujo',
  'studio_Pops Inc.',
  'genre_Magic',
  'genre_Gakken',
  'studio_Kachidoki Studio',
  'genre_Slice of Life',
  'genre_Shueisha',
  'studio_Sunrise',
  'genre_Yomiko Advertising',
  'genre_Obtain Future',
  'genre_Shogakukan Productions',
  'genre_TOHO animation',
  'genre_Sony Music Communications',
  'genre_Lantis',
  'genre_AT-X',
  'Type_TV',
  'studio_Bones',
  'genre_Hakusensha',
  'genre_Kodansha',
  'genre_Shogakukan',
  'genre_TV Asahi',
  'genre_Sotsu',
  'genre_Genco',
  'genre_Frontier Works',
  'genre_Dentsu',
  'genre_Space Shower Music',
  'genre_Good Smile Company',
  'genre_Rakuonsha',
  'genre_Harem',
  'studio_Opera House',
  'genre_Showgate',
  'studio_Studio Deen

In [73]:
# From correlated features with TARGET, select their top 3 correlated features
selected_features = correlated_columns["Rating"]
for feature in selected_features.copy():
    correlated_correlated_features = high_corr[feature].abs().sort_values(ascending=False)

    features_to_select = correlated_correlated_features[correlated_correlated_features < 90][:3].index.tolist()
    selected_features.extend(features_to_select)
df = df.loc[:, np.unique(selected_features)]


In [74]:
df['Synopsis']=synopsis
df['Title']=Title
df

C:\Users\hp\AppData\Local\Temp\ipykernel_13800\2643210191.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Synopsis']=synopsis
C:\Users\hp\AppData\Local\Temp\ipykernel_13800\2643210191.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Title']=Title


,Rating,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_TV,genre_12 Diary Holders,genre_ABC Animation,genre_ASCII Media Works,genre_AT-X,...,studio_WAO World,studio_Wit Studio,"studio_Yamamura Animation, Inc.",studio_domerica,studio_dwarf,studio_feel.,studio_pH Studio,studio_ufotable,Synopsis,Title
0,8.81,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.00020464750237225257, 0.005315192563449676,...","[0.019310925796162337, -0.011677875027089613]"
1,8.41,1,0,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.00959832645049773, -0.03554391090775181, -0...","[0.006436975265387446, -0.003892625009029871, ..."
2,8.31,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0010812541723046734, 0.005207814935905238, ...",[-0.009230552852386609]
3,7.34,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.005169035755071408, -0.15829741876487705, -...","[0.01138263581863915, 0.035894299702097975, -0..."
4,7.04,0,0,0,0,1,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.06922955825784871, -0.045365353410126145, ...","[0.009988797440504035, 0.015221555911314985, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15030,NaN,0,0,1,0,0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0017630417576237093, 0.059579316792743545, ...","[-0.0034692702029133216, 0.14585203697206453, ..."
15031,5.29,0,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.10645002868841402, 0.0021177566293317795, -...","[0.00932627295878774, 0.004464440162337269, 0...."
15037,NaN,1,0,0,0,0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.011464242838943999, 0.02087888310973843, 0....","[0.006666698632761836, 0.05224411659874022, -0..."
15042,6.91,0,0,1,0,0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0.13591757319359618, 0.09161201885647395, 0.0...","[0.008931750229749014, 0.18975995294749737, 0...."


In [75]:
def average_vectors(vectors):
    return np.mean(vectors, axis=0)

In [76]:
df["Title"] = df["Title"].apply(average_vectors)

In [77]:
df["Synopsis"] = df["Synopsis"].apply(average_vectors)

c:\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [78]:
df = df.dropna(subset=['Rating'])


In [79]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


# Séparation des données en entrée et sortie
X =df.drop(['Rating'], axis=1)
y = df['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [80]:
# Entraîner le modèle
from sklearn.ensemble import HistGradientBoostingRegressor
model = HistGradientBoostingRegressor()
model.fit(X_train, y_train)



HistGradientBoostingRegressor()

In [81]:
import pickle
filename = 'model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [82]:
y_pred= model.predict(X_test)

In [83]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(" R2 : ", r2)
print(" mse : ", mse)

 R2 :  0.5006849467801544
 mse :  0.6782175191439588


In [84]:
df

,Rating,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_TV,genre_12 Diary Holders,genre_ABC Animation,genre_ASCII Media Works,genre_AT-X,...,studio_WAO World,studio_Wit Studio,"studio_Yamamura Animation, Inc.",studio_domerica,studio_dwarf,studio_feel.,studio_pH Studio,studio_ufotable,Synopsis,Title
0,8.81,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.011565,0.003817
1,8.41,1,0,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.011989,0.065535
2,8.31,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.015045,-0.009231
3,7.34,0,0,0,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.018808,0.007872
4,7.04,0,0,0,0,1,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.025081,0.010364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15015,5.52,0,1,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084490,0.032833
15016,5.52,0,1,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006456,0.021504
15031,5.29,0,0,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049371,0.035174
15042,6.91,0,0,1,0,0,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000710,0.064726


In [94]:
noms_colonnes = df.columns
data = pd.DataFrame(columns=noms_colonnes,index=[1])

In [86]:
data= data.drop(['Rating'], axis=1)


In [95]:
data.to_csv('data.csv')

In [88]:
def fill_data(valeurs_entree, synopsis, titre, data):
    for valeur_entree in valeurs_entree:
        for i in data.columns:
            parts = i.rsplit('_', 1)
            if len(parts) == 2 and valeur_entree == parts[1]:
                row_index = 0  # assume there is only one row in the DataFrame
                column_name = i
                data.at[row_index, column_name] = 1

    data['Synopsis'] = synopsis
    data['Title'] = titre

    return data

In [89]:
valeur_entree= ['nnn', 'gggg', 'jj', 'nn', 'fggg', 'ggcfwfdw']

In [90]:
text="je suis allée au champ"

In [92]:
dii = pd.read_csv('data.csv')

In [93]:
dii

,Unnamed: 0,Type_Movie,Type_Music,Type_ONA,Type_OVA,Type_TV,genre_12 Diary Holders,genre_ABC Animation,genre_ASCII Media Works,genre_AT-X,...,studio_WAO World,studio_Wit Studio,"studio_Yamamura Animation, Inc.",studio_domerica,studio_dwarf,studio_feel.,studio_pH Studio,studio_ufotable,Synopsis,Title
